This is a network that does feature-extraction on a network trained on the ImageNet dataset. After a few image batches it is approximately 95% accurate on the test data and can get up to 97.5% accuracy after a few more iterations.

In [ ]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict

In [ ]:
def save_model(checkpoint):
    torch.save(checkpoint, 'catdog_classifier.pth')

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device ", device)

In [ ]:
# Load the data
data_dir = 'Cat_Dog_data'

# the pretrained network was trained with means [0.485, 0.456, 0.406] and std. devs. [0.229, 0.224, 0.225]
# and expects images cropped to 224x224 pixels
train_transforms = transforms.Compose([transforms.RandomRotation(30), 
                                       transforms.RandomResizedCrop(224), 
                                       transforms.RandomHorizontalFlip(), 
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255), 
                                      transforms.CenterCrop(224), 
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)
print(f"{len(trainloader)} images loaded into trainloader")
print(f"{len(testloader)} images loaded into testloader")

In [ ]:
# Hyperparameters
input_layer = 2048
hidden_layer = 512
output_layer = 2
hidden_layer_dropout = 0.25
epochs = 1

In [ ]:
# Setup the pretrained network and the classifier for training
model = models.resnet50(pretrained=True)

# freeze the pretained model's parameters
for param in model.parameters():
    param.requires_grad = False

# define my classifier that will be trained on the cat and dog images
cat_dog_classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(input_layer, hidden_layer)),
    ('relu', nn.ReLU()),
    ('dropout', nn.Dropout(p=hidden_layer_dropout)),
    ('fc2', nn.Linear(hidden_layer, output_layer)),
    ('output', nn.LogSoftmax(dim=1))
]))

model.fc = cat_dog_classifier
print("Model with new classifier ", model)

model.to(device)

In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.01)
steps = 0
saved = False
train_loss = 0

print("Starting training...")
train_losses, test_losses = [], []
for e in range(epochs):
    print(f"Training epoch: {e + 1}")
    for images, labels in trainloader:
        steps += 1
        images, labels = images.to(device), labels.to(device)
        
        # forward pass, backpropagate, step
        optimizer.zero_grad()
        outputs = model.forward(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
        # test the model every 3 image batches and save it if accuracy on test data is over 97.5%
        if steps % 3 == 0: 
            test_loss = 0
            accuracy = 0
            model.eval()

            with torch.no_grad():
                for images, labels in testloader:
                    images, labels = images.to(device), labels.to(device)

                    log_probs = model(images)
                    probs = torch.exp(log_probs)

                    loss = criterion(log_probs, labels)
                    test_loss += loss.item()

                    top_p, top_class = probs.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))

            model.train()
            train_losses.append(train_loss / len(trainloader)) 
            test_losses.append(test_loss / len(testloader))
            
            print("Epoch: {}/{}.. ".format(e + 1, epochs),
                  "Training Loss: {:.3f}.. ".format(train_loss / len(trainloader)),  # avg train loss over 3 batches
                  "Test Loss: {:.3f}.. ".format(test_loss / len(testloader)), # avg test loss over 3 batches
                  "Test Accuracy: {:.3f}".format(accuracy / len(testloader))) # avg accuracy of model on test data after training on 3 batches
            
            train_loss = 0
            if accuracy / len(testloader) > 0.975: 
                checkpoint = {'input_size': input_layer, 
                              'output_size': output_layer, 
                              'state_dict': model.state_dict()}
                save_model(checkpoint)
                saved = True
                print("Saved cat dog classifier. Breaking.")
                break;
                
print("Done training.")
if not saved:
    save_model({'input_size': input_layer, 'output_size': output_layer, 'state_dict': model.state_dict()})

Sample training run:
```
Epoch: 1/1..  Training Loss: 0.094..  Test Loss: 18.271..  Test Accuracy: 0.488
Epoch: 1/1..  Training Loss: 0.080..  Test Loss: 4.909..  Test Accuracy: 0.512
Epoch: 1/1..  Training Loss: 0.033..  Test Loss: 1.656..  Test Accuracy: 0.495
Epoch: 1/1..  Training Loss: 0.020..  Test Loss: 0.222..  Test Accuracy: 0.911
Epoch: 1/1..  Training Loss: 0.006..  Test Loss: 0.186..  Test Accuracy: 0.930
Epoch: 1/1..  Training Loss: 0.004..  Test Loss: 0.180..  Test Accuracy: 0.933
Epoch: 1/1..  Training Loss: 0.004..  Test Loss: 0.105..  Test Accuracy: 0.966
Epoch: 1/1..  Training Loss: 0.003..  Test Loss: 0.102..  Test Accuracy: 0.977
```